# 1-1. 데이터 차원 여부 확인

## 1. 변수(특징) 개수 확인

In [ ]:
# 1)

print(f"데이터 샘플 수 (행): {data.shape[0]}")
print(f"데이터 변수 수 (열): {data.shape[1]}")

if data.shape[1] > 20:  # 변수 20개 이상을 고차원으로 간주
    print("데이터는 고차원 데이터입니다.")
else:
    print("데이터는 저차원 데이터입니다.")


In [ ]:
# 2)

import pandas as pd

# 예제 데이터
data = {
    'Feature1': [1, 2, 3],
    'Feature2': [4, 5, 6],
    'Feature3': [7, 8, 9],
    'Feature4': [10, 11, 12],
    'Feature5': [13, 14, 15]
}
df = pd.DataFrame(data)

# 데이터 크기 출력
print(f"데이터 크기: {df.shape}")  # (샘플 수, 특성 수)

# 샘플 대비 특성 비율 확인
num_samples, num_features = df.shape
if num_features > num_samples:
    print("고차원 데이터입니다.")
else:
    print("고차원이 아닙니다.")


## 상관관계 분석

In [ ]:
import pandas as pd

# 상관행렬 계산
corr_matrix = data.corr()

# 상관행렬 시각화
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title("Feature Correlation Matrix")
plt.show()

# 상관계수 확인
high_corr = (abs(corr_matrix) > 0.8).sum().sum() - len(corr_matrix)
print(f"상관계수가 0.8 이상인 변수 쌍 개수: {high_corr}")


## 샘플 밀도

-> 샘플이 전체 공간에서 희소하면 고차원

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

# 샘플 간 유클리드 거리 계산
distances = euclidean_distances(data)
avg_distance = np.mean(distances)

print(f"샘플 간 평균 거리: {avg_distance:.2f}")
if avg_distance > 1.0:
    print("고차원 데이터로 인해 희소성이 나타날 가능성이 있습니다.")
else:
    print("데이터가 고차원적이지 않습니다.")


*
군집의 밀도와 모양이 구형이 아닌 경우, 밀도 기반 클러스터링(DBSCAN)을 사용하여 분석.
DBSCAN은 구형이 아닌 군집도 효과적으로 탐지합니다.
DBSCAN 결과와 비교하여 구형 클러스터링이 적절한지 판단.

# 1-2. 군집형태가 구형인지 파악



* 구형 : k-means

* 아니면 DBSCAN

In [ ]:
# 시각화 - 2D / 3D 시각화

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# PCA로 차원 축소
pca = PCA(n_components=2)
pca_data = pca.fit_transform(data)

# 클러스터링 결과 시각화 (예: K-Means 결과)
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=clusters, cmap='viridis')
plt.title("Cluster Visualization (PCA Reduced)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label="Cluster")
plt.show()

In [ ]:
# 시각화 - Pairplot 시각화

import seaborn as sns
sns.pairplot(data.assign(Cluster=clusters), hue="Cluster", diag_kind="kde")
plt.show()


In [ ]:
# 2. 군집 거리 분석-  Silhouette Score

from sklearn.metrics import silhouette_score

sil_score = silhouette_score(data, clusters)
print(f"Silhouette Score: {sil_score:.2f}")

In [ ]:
# 2. 군집 내 분산 비교

import numpy as np

# 각 군집의 중심과 데이터 거리 계산
centroids = kmeans.cluster_centers_
distances = [np.linalg.norm(data[clusters == i] - centroids[i], axis=1) for i in range(len(centroids))]

# 평균 거리 출력
avg_distances = [np.mean(dist) for dist in distances]
print(f"군집별 평균 거리: {avg_distances}")


In [ ]:
#  Ellipticity 검사(이거 쓰면 빠르지 않을까..??) -> 타원형 군집의 중심과의 거리가 일정하지 않다면 구형x

from scipy.spatial.distance import mahalanobis
import numpy as np

# 데이터 공분산 행렬
cov_matrix = np.cov(data.T)

# Mahalanobis 거리 계산 (예: 첫 군집)
center = centroids[0]
mahalanobis_distances = [mahalanobis(point, center, np.linalg.inv(cov_matrix)) for point in data[clusters == 0]]

# 평균 거리 출력
print(f"Mahalanobis Distance 평균: {np.mean(mahalanobis_distances):.2f}")


# 2-1 데이터가 저차원??

In [ ]:
# 주의 사항 1. 데이터 스케일링하기

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)


In [ ]:
# 주의 사항 2. 이상치 확인하기

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
from sklearn.preprocessing import StandardScaler

# ... (previous code)

# 주의 사항 2. 이상치 확인하기
def detect_outliers(df, features):
    outlier_indices = []

    for col in features:
        # IQR 기반 이상치 탐지
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR

        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)

    outlier_indices = sorted(list(set(outlier_indices)))
    return outlier_indices

# 이상치 탐지 실행 (모든 열에 대해)
outlier_indices = detect_outliers(df, df.columns)

# 이상치 출력
print("이상치 인덱스:", outlier_indices)

# 이상치 시각화 (박스플롯)
plt.figure(figsize=(10, 6))
sns.boxplot(data=df)
plt.title("Boxplot of Features")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 이상치 제거 예시

import numpy as np

# IQR을 활용한 이상치 제거 예시
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
filtered_data = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]

### 클러스터링 바로 적용

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Elbow Method로 최적 k 찾기
inertia = []
k_values = range(1, 10)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

# Elbow Plot
plt.plot(k_values, inertia, marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.show()


In [ ]:
# 데이터 표준화
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[['소득', '소비']])

# K-Means 클러스터링
kmeans = KMeans(n_clusters=3, random_state=42)
df['클러스터'] = kmeans.fit_predict(scaled_data)

# 결과 출력
print(df)

# 클러스터링 시각화
plt.figure(figsize=(8, 6))
for cluster in df['클러스터'].unique():
    cluster_data = df[df['클러스터'] == cluster]
    plt.scatter(cluster_data['소득'], cluster_data['소비'], label=f"Cluster {cluster}")

plt.xlabel('소득')
plt.ylabel('소비')
plt.title('K-Means Clustering')
plt.legend()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

# DBSCAN 클러스터링
dbscan = DBSCAN(eps=0.5, min_samples=5)  # epsilon과 min_samples는 데이터에 맞게 조정
db_clusters = dbscan.fit_predict(scaled_data)

# 클러스터링 결과 시각화 (2D 데이터인 경우)
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=db_clusters, cmap='viridis')
plt.title("DBSCAN Clustering")
plt.show()


# 2-2: 만약 데이터 고차원 PCA

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 1. 데이터 로드
# 데이터는 이미 전처리된 상태라고 가정
data = pd.read_csv("high_dimensional_data.csv")

# 2. 데이터 스케일링
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# 3. PCA를 이용한 차원 축소
# 주성분 수는 누적 설명력이 90% 이상이 되는 값으로 선택
pca = PCA()
pca_data = pca.fit_transform(scaled_data)

# 누적 설명력 계산 및 시각화
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()

plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.6, label='Individual Explained Variance')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Cumulative Explained Variance', color='red')
plt.title("Explained Variance by PCA Components")
plt.xlabel("Number of Components")
plt.ylabel("Explained Variance Ratio")
plt.legend(loc='best')
plt.grid()
plt.show()

# 주성분 수 결정 (90% 이상의 설명력을 가지는 최소 차원 수)
optimal_components = (cumulative_variance >= 0.90).argmax() + 1
print(f"Optimal number of components: {optimal_components}")

# PCA 변환 (최적 차원으로 축소)
pca = PCA(n_components=optimal_components)
reduced_data = pca.fit_transform(scaled_data)

# 4. 클러스터링
## a. K-Means 클러스터링
kmeans = KMeans(n_clusters=4, random_state=42)  # 군집 수는 예시로 4개 설정
kmeans_labels = kmeans.fit_predict(reduced_data)

## b. DBSCAN 클러스터링
dbscan = DBSCAN(eps=0.5, min_samples=5)  # epsilon과 min_samples는 데이터에 따라 조정
dbscan_labels = dbscan.fit_predict(reduced_data)

# 5. 결과 평가
## Silhouette Score 계산
kmeans_sil_score = silhouette_score(reduced_data, kmeans_labels)
print(f"Silhouette Score (K-Means): {kmeans_sil_score:.2f}")

if len(set(dbscan_labels)) > 1:  # DBSCAN은 -1(노이즈) 포함 가능
    dbscan_sil_score = silhouette_score(reduced_data, dbscan_labels)
    print(f"Silhouette Score (DBSCAN): {dbscan_sil_score:.2f}")
else:
    print("DBSCAN 결과에 군집이 하나만 있거나 노이즈가 많습니다.")

# 6. 시각화 (2D PCA 데이터 기준)
plt.figure(figsize=(12, 5))

# K-Means 시각화
plt.subplot(1, 2, 1)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=kmeans_labels, cmap='viridis')
plt.title("K-Means Clustering")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")

# DBSCAN 시각화
plt.subplot(1, 2, 2)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=dbscan_labels, cmap='viridis')
plt.title("DBSCAN Clustering")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")

plt.tight_layout()
plt.show()


### 코드 구성 설명

* 스케일링: PCA와 클러스터링을 정확히 수행하기 위해 데이터 스케일링 필수.

* PCA 변환:
모든 주성분의 누적 설명력을 확인하고, 90% 이상 설명력을 가진 최소 차원 수를 결정.
PCA로 데이터를 축소.

* 클러스터링:
K-Means: 사전에 군집 수를 지정. 결과를 Silhouette Score로 평가.

* DBSCAN: 밀도 기반 클러스터링. 노이즈를 포함한 결과를 생성.



* 결과 평가: Silhouette Score를 통해 각 클러스터링의 품질 평가.

* 시각화:
PCA로 축소한 2D 데이터를 기준으로 클러스터링 결과를 시각화.
결과 해석

* PCA의 주성분 개수: 적절한 차원을 선택해 차원 축소.
클러스터링 품질: Silhouette Score로 클러스터링의 적합성 평가.

* 군집 결과 시각화: 군집 구조를 직관적으로 이해.